In [1]:
import multiprocessing
from multiprocessing import Pool

import pandas as pd
from PKM_multiprocess import ParallelCompute_PKM as PC_PKM

result=PC_PKM(-20)
print(result)

Тепла в ГВТО хватает на теплофикацию
Не сходится баланс камеры сгорания 42.24097749603402 5.396572078097961e-12 42.24097749602862
Рассчитана пиковая ГТУ, расход воздуха: 946.3198376649956
Время 1 итерации расчета КУ+ТУ:---  6.9 сек. ---
Время 2 итерации расчета КУ+ТУ:---  18.3 сек. ---
Время 3 итерации расчета КУ+ТУ:---  22.2 сек. ---
Время 4 итерации расчета КУ+ТУ:---  68.8 сек. ---
Время 5 итерации расчета КУ+ТУ:---  89.9 сек. ---
Время 6 итерации расчета КУ+ТУ:---  108.0 сек. ---
Время 7 итерации расчета КУ+ТУ:---  125.6 сек. ---
Время 8 итерации расчета КУ+ТУ:---  138.0 сек. ---
Время 9 итерации расчета КУ+ТУ:---  146.6 сек. ---
Время 10 итерации расчета КУ+ТУ:---  157.3 сек. ---
Время 11 итерации расчета КУ+ТУ:---  159.4 сек. ---
Время 12 итерации расчета КУ+ТУ:---  161.2 сек. ---
Время 13 итерации расчета КУ+ТУ:---  166.5 сек. ---
Время 14 итерации расчета КУ+ТУ:---  168.3 сек. ---
Расчет КУ+ПТУ окончен.
Максимальная погрешность определения расходов при расчете КУ+ПТУ 0.0
Pnd_it 

In [2]:
# Импорт библиотек
import os
import time

import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from calculate_CCGT_PKM import Calculate_CCGT_PKM_iter
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
accumulation = pd.read_excel("blocks.xlsx", sheet_name="accumulation", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWIN", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWIN", "T"]
water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################

# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 4
Maxiterations_cotel = 2
Maxiterations_turbine = 15
Iter_pkm = 6


# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True


# Расчет для работы с теплофикацией
# Teplo = int(False)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = list(
    map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures)
)


#####################Максимов#####################
water_streams.at["SWIN", "H"] = water.p_t(1, water_streams.at["SWIN", "T"])["h"]
water_streams.at["SWOUT", "H"] = water.p_t(1, water_streams.at["SWOUT", "T"])["h"]

# Время зарядки/разрядки, часы
time_ac = 4
# Время ожидания, часы
time_jdat = 12
# Конструкция аккумулятора
constr = {
    "Diametr": 20,
    "kolichestvo": 8,
    "Visota": 20,
    "lambda_min_vata": 0.045,
    "delta_min_vata": 0.01,
}
PKM_zaryad = True
PKM_razryad = False
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

pkm_pgu_tol = 10**-2

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
    constr,
    time_jdat,
]


########################ОГРАНИЧЕНИЯ НА РАБОТУ ПГУ########################
# ЕСЛИ ТЕПЛОФИКАЦИЯ (из документов по ПГУ-220Т)
# Gк_мин=4,44 кг/с вроде
# Nтурбины мин = 17,6 МВт (примерно 25%)
# Максимальноа давление в отборах СП 0,245, 0,198 МПа (Теплофикационная паровая турбина Т-63/76-8.8 для серии ПГУ-230)
# ДЛЯ ВСЕХ РЕЖИМОВ (Из Трухния по ПГУ-450Т)
# Gвд_мин=25% от номинала
# Gнд_мин = 50% от номинала


########РАСЧЕТ расхода пара на ПКМ из условия заполнения хранилища################
########РАСЧЕТ мощности ГТУ из условия тепловой мощности и ################

################Расчет минимальной нагрузки ГТУ при остальных нормальных условиях############

# Max_iterations_minimum = 20

# n_GTU_it = [0]
# Delta_n_GTU = 100
# if Сalculate_minimum == True:
#     gas_streams.loc["GTU-PEVD", "G"] = gas_streams.loc["GTU-KU", "G"]
#     n_GTU = GTU_input.at["n", 1]
#     start_time = time.time()
#     Delta_min = 0
#     n_GTU_it.append(round(n_GTU, 5))
#     for i in range(Max_iterations_minimum):
#         print("n_GTU:", n_GTU_it)
#         if i < 3:
#             (
#                 New_iterations_KU_TU,
#                 New_iterations_cotel,
#                 New_iterations_turbine,
#                 New_Iter_pkm,
#             ) = (2, 2, 15, 4)
#         else:
#             print("Delta_n_GTU: ", Delta_n_GTU)
#             (
#                 New_iterations_KU_TU,
#                 New_iterations_cotel,
#                 New_iterations_turbine,
#                 New_Iter_pkm,
#             ) = (
#                 Maxiterations_KU_TU,
#                 Maxiterations_cotel,
#                 Maxiterations_turbine,
#                 Iter_pkm,
#             )

#         gas_streams = Calculate_CCGT_PKM_iter(
#             arguments_all_it, New_Iter_pkm, pkm_pgu_tol
#         )
#         Delt_Gcnd = 100
#         Delt_Nturb = 100
#         Delt_Gcnd = (water_streams.at["INKOND", "G"] - 4.44) / 4.44
#         Delt_Nturb = (electric.at["Turbine", "Ni"] - 17.6) / 17.6
#         Delt_Gvd = (
#             water_streams.at["PEVD-DROSVD", "G"] / water_streams0.at["PEVD-DROSVD", "G"]
#             - 0.25
#         ) / 0.25
#         Delt_Gnd = (
#             water_streams.at["PPND-DROSND", "G"] / water_streams0.at["PPND-DROSND", "G"]
#             - 0.5
#         ) / 0.5

#         arguments_all_it[0], arguments_all_it[1], arguments_all_it[2] = (
#             New_iterations_KU_TU,
#             New_iterations_cotel,
#             New_iterations_turbine,
#         )

#         Delta_n_GTU = abs((n_GTU_it[-1] - n_GTU_it[-2]) / n_GTU_it[-1] * 100)
#         Delta_min = min(Delt_Gcnd, Delt_Nturb, Delt_Gvd, Delt_Gnd)
#         n_GTU = n_GTU - Delta_min / 5
#         GTU_input.at["n", 1] = n_GTU
#         n_GTU_it.append(round(n_GTU, 5))

#         print(
#             f"Время {i+1} итерации расчета мощности ГТУ при ПГУ с ПКМ: --- {round((time.time() - start_time), 1)} сек. ---"
#         )
#         print(f"Отклонение от ограничения минимальное равно {Delta_min}")

#         if n_GTU == 1 and Delta_min < 0:
#             print("Мощность ГТУ 100% и расход пара все еще слишком мал")

#         if abs(Delta_min) < Calctolerance and Delta_n_GTU < Calctolerance:
#             arguments_all_it[0], arguments_all_it[1], arguments_all_it[2] = (
#                 Maxiterations_KU_TU,
#                 Maxiterations_cotel,
#                 Maxiterations_turbine,
#             )
#             n_GTU_it.append(round(n_GTU, 5))
#             gas_streams = Calculate_CCGT_PKM_iter(
#                 arguments_all_it, New_Iter_pkm, pkm_pgu_tol
#             )
#             print(f"Отклонение от ограничения минимальное равно {Delta_min}")
#             print(f"Относительная мощность ГТУ равна {n_GTU}")
#             print(
#                 f"fin минимальная мощность ПГУ:--- {round((time.time() - start_time), 1)} сек. ---"
#             )
#             print("n_GTU_it", n_GTU_it)
#             break
#         if i == Max_iterations_minimum - 1:
#             print(
#                 "Достигнуто максимальное количество итераций минимального расхода в ПГУ",
#                 i + 1,
#             )
#             print("n_GTU_it", n_GTU_it)

In [3]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,5.0,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,542.1,0.1,958.869199,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
REF-SMESH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-PEVD,NaN,NaN,NaN,503.8,NaN,NaN,NaN,NaN,NaN
PEVD-IVD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IVD-EVD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EVD-PPND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PPND-IND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IND-GPK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GPK-out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
water_streams

,T,P,H,G,S,X
AIR,5.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,511.500000,8.407000,3423.209978,63.400000,NaN,NaN
IVD-PEVD,301.297556,8.746000,2747.104023,63.400000,NaN,NaN
EVD-IVD,298.800000,8.746000,1337.948265,63.400000,NaN,NaN
PEN-EVD,166.500000,8.746000,708.319849,63.400000,NaN,NaN
BND-PEN,164.738361,0.696400,696.095567,63.400000,NaN,NaN
PPND-DROSND,211.600000,0.691000,2871.481837,14.760000,NaN,NaN
IND-PPND,164.738361,0.696400,2762.534980,14.760000,NaN,NaN
GPK-IND,164.600000,0.696400,695.493701,78.160000,NaN,NaN
GPK-REC,164.600000,0.696400,695.493701,82.910000,NaN,NaN


In [5]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COOL-HTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HTS-HTSCOOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HTSCOOL-Separ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Separ-SGaccum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SGaccum-Separacc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Separacc-Sepout,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sepout-COMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sepout-GTU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AIR-COMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
electric

,Ni,N,KPDm,KPD
PEN,NaN,NaN,NaN,NaN
CVD,NaN,NaN,NaN,NaN
CND,NaN,NaN,NaN,NaN
GTU,NaN,NaN,NaN,NaN
DK,NaN,NaN,NaN,NaN
Turbine,NaN,NaN,NaN,NaN
Tots1,NaN,NaN,NaN,NaN
Tots2,NaN,NaN,NaN,NaN
Tots3,NaN,NaN,NaN,NaN
Tots4,NaN,NaN,NaN,NaN


In [7]:
heaters

,Qw,Qg,KPD
PEVD,NaN,NaN,NaN
IVD,NaN,NaN,NaN
EVD,NaN,NaN,NaN
PPND,NaN,NaN,NaN
IND,NaN,NaN,NaN
GPK,NaN,NaN,NaN
SP2,NaN,NaN,NaN
SP1,NaN,NaN,NaN
OD,NaN,NaN,NaN
Strans,NaN,NaN,NaN


In [8]:
accumulation

,Qw,T,V
ASW,NaN,NaN,NaN
PKM,NaN,250.0,10000.0


In [ ]:
PKM_zaryad = False
PKM_razryad = True

Maxiterations_KU_TU = 20
Maxiterations_cotel = 5
Maxiterations_turbine = 20

n_GTU = 1
GTU_input.at["n", 1] = n_GTU

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
    constr,
    time_jdat,
]
New_Iter_pkm = 20
pkm_pgu_tol = 10**-2
CCGT = Calculate_CCGT_PKM_iter(arguments_all_it, New_Iter_pkm, pkm_pgu_tol)

Тепла в ГВТО хватает на теплофикацию
Не сходится баланс камеры сгорания 41.95839694385042 5.382361223382759e-12 41.95839694384503
Рассчитана пиковая ГТУ, расход воздуха: 951.5353556149116
Время 1 итерации расчета КУ+ТУ:---  12.2 сек. ---
Время 2 итерации расчета КУ+ТУ:---  23.7 сек. ---
Время 3 итерации расчета КУ+ТУ:---  27.8 сек. ---
Время 4 итерации расчета КУ+ТУ:---  56.9 сек. ---
Время 5 итерации расчета КУ+ТУ:---  78.0 сек. ---
Время 6 итерации расчета КУ+ТУ:---  95.6 сек. ---
Время 7 итерации расчета КУ+ТУ:---  108.9 сек. ---
Время 8 итерации расчета КУ+ТУ:---  121.8 сек. ---


In [ ]:
arguments_all_it

In [ ]:
GTU_input

In [ ]:
gas_streams

In [ ]:
water_streams

In [ ]:
syngas_streams

In [ ]:
electric

In [ ]:
heaters

In [ ]:
accumulation